In [ ]:
### Flynn's Taxonomy 

Characterize machines by number of instruction streams and data streams
  * Defined in 1972.  Still common practice.
  * A little too restrictive, but a starting place

